<a href="https://colab.research.google.com/github/WebiksInc/data-explorer/blob/add-stop-word/function_save_new_try_Interactive_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pickle

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import IPython.display
from IPython.display import display, clear_output
import os
import warnings
warnings.filterwarnings('ignore')

path_main = r'C:\corpus\for display'
os.chdir(path_main)
process_data = False

###voila

# Comparison of datasets

In [ ]:
def number_lines(df:pd.Series) -> int:
    num_line = df.shape[0]
    return num_line

def number_words(df:pd.Series) -> int:
    return df[df.notnull()].astype("string").str.split().apply(len).sum()

def create_folder(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
def plot_shape_all_data(names):
    if process_data:
        result = {}
        fx = lambda x: [number_words(x),number_lines(x)]
        for index in names:
            if  index == 'ben yehuda':
                df = pd.read_csv(r'C:\corpus\ben yehuda\df.csv')
                result[index] = fx(df['line'])
            elif index == 'think IL':
                df = pd.read_csv(r'C:\corpus\think il\df think IL token.tsv',sep = '\t')
                df = df.rename(columns={'text':'line','auther':'author'})
                result[index] = fx(df['line'])
#             elif index == 'NER':
#                 df = pd.read_csv(r'C:\corpus\NER Naama Ben-Mordecai\df.csv',sep = '\t')
#                 result[index] = fx(df['line'])
            elif index == 'paraShoot':
                df = pd.read_csv('C:\corpus\ParaShoot\df.csv',sep = '\t')
                df = df.rename(columns = {'question':'line'})
                result[index] = fx(df['line'])
            elif index == 'Hebrew-Sentiment':
                df = pd.read_csv('C:\corpus\Hebrew-Sentiment-Data\dev.tsv',sep = '\t')
                df = df.rename(columns = {'comment':'line'})
                result[index] = fx(df['line'])
            elif index == 'HeBERT Emotion Recognition':
                df = pd.read_csv(r'C:\corpus\HeBERT Emotion Recognition\test_insample_tagged_raw_to_publish.csv')
                df = df.rename(columns = {'talkbacks':'line'})
                result[index] = fx(df['line'])
            elif index == 'wikipedia':
                df = pd.read_csv(r'C:\corpus\subset wiki\ he df.csv',sep = '\t')
                df = df.rename(columns = {'content':'line'})
                result[index] = fx(df['line'])
            elif index == 'mila':
                for root, dirs, files in os.walk(r'C:\corpus\mila'):
                    for file in files:
                        if file.endswith('.csv'):
                            df = pd.read_csv('C:\\corpus\\mila\\'+ file,sep = '\t')
                            df = df.rename(columns = {'text':'line'})
                            result[file.split('.')[0]] = fx(df['line'])
        result = pd.DataFrame.from_dict(result, orient='index').reset_index()
        result.columns = ['Name Database','Number of Words','Number of Lines']
        create_folder('main')
        result.to_csv('main/result.csv')
    else:
        result = pd.read_csv('main/result.csv')
        
        fig = make_subplots(rows=1, cols=2,subplot_titles=("Number of Words", "Number of Lines"))
        fig.add_trace(
            go.Bar(x=result['Number of Words'], y=result['Name Database'],orientation='h'),
            row=1, col=1
        )
        fig.update_xaxes(type="log", row=1, col=1)

        fig.add_trace(
            go.Bar(x=result['Number of Lines'], y=result['Name Database'],orientation='h'),
            row=1, col=2
        )
        fig.update_xaxes(type="log", row=1, col=2)
        fig.update_layout(showlegend=False)
        fig.show()
process_data = False
# process_data = True
plot_shape_all_data(['ben yehuda','think IL','NER','paraShoot','Hebrew-Sentiment','HeBERT Emotion Recognition','wikipedia','mila'])

In [ ]:
# function

def number_lines(df:pd.Series) -> int:
    num_line = df.shape[0]
    return num_line



def number_word_in_line(df:pd.Series) -> dict:
    result = df.apply(lambda x: len(x.split(' ')))
    # fig = px.histogram(result, x="line")
    return {'mean' : np.round(result.mean()),
            'median' : result.median()}


def char_freq(df:pd.Series,plot = False) -> px.bar:
    from collections import Counter
    x = df.tolist()
    x = Counter(''.join(x))
    x = dict(sorted(x.items(), key=lambda item: item[1]))
    x.pop('\u200f',None) #TODO: it's ok to delete this char?
    return x
    if plot:
        fig = px.bar(x=x.values(), y=x.keys(), height=1500,width=800)
        return fig
    else:
        return x


def ngram(df:pd.Series, ngram:int) -> dict:
    from sklearn.feature_extraction.text import CountVectorizer
    df = df.replace('[^\u0590-\u05fe a-zA-Z]',' ',regex=True)
    # Preparations
    vec = CountVectorizer(ngram_range=(ngram, ngram)).fit(df)
    bag_of_words = vec.transform(df)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    dat = pd.DataFrame(words_freq, columns=['word','frequency'])
    dat = dat.sort_values('frequency',ascending = False)
    one_apper = dat[dat['frequency'] == 1]

    #gini
    sorted_x = np.sort(dat['frequency'])
    n = len(sorted_x)
    cumx = np.cumsum(sorted_x, dtype=float)
    gini = ((n + 1 - 2 * np.sum(cumx) / cumx[-1])/ n)

    return({'shape unique words' : dat.shape[0],
                'shape appeared once' : one_apper.shape[0],
                'Percent appeared once' : one_apper.shape[0]/dat.shape[0],
                'top unqiue words' : dat,
#                 'tail unique words' : one_apper.sample(number_return),
                'type-token ratio' : dat.shape[0]/dat['frequency'].sum(),
                'Number Of Words' : dat['frequency'].sum(),
                'gini' : gini
                }
        )



def identity_duplicate_line(df:pd.Series) -> int:
    return df.duplicated().sum()

def topic_model(df:pd.Series,k = 4) -> list:
    # https://github.com/ashishsalunkhe/Topic-Modeling-using-LDA-and-K-Means-Clustering/blob/master/newsgroup.ipynb
    #important notebook
    from sklearn.feature_extraction.text import TfidfVectorizer
    text_content = df
    vector = TfidfVectorizer()
    tfidf = vector.fit_transform(text_content)

    terms = vector.get_feature_names()

    from sklearn.decomposition import NMF
    result = []

    nmf  = NMF(n_components = k)
    nmf.fit(tfidf)
    for i in range(0,k):
        word_list=[]
        # print("Topic%d:"% i)
        for j in nmf.components_.argsort()[i,-16:-1]:
            word_list.append(terms[j])
        result.append(word_list)
    return result

def co_occurrence(df:pd.Series,dirc: str) -> pd.DataFrame:
    from sklearn.feature_extraction.text import CountVectorizer

    temp = df.apply(lambda x: x.split('.')).explode()
    count_model = CountVectorizer(ngram_range=(1,1))
    X = count_model.fit_transform(temp)
    Xc = (X.T * X)
    Xc.setdiag(0)
    co_oc = pd.DataFrame(Xc.todense(), 
                        columns=count_model.get_feature_names(), 
                        index=count_model.get_feature_names())
    word_co = load_pickle(dirc,'ngram 1.pickle')['top unqiue words']['word'].head().to_list()
    result = {}
    for word in word_co:
        temp = co_oc[word].nlargest(10).to_frame().reset_index()
        temp.columns = ['number','word']
        temp = temp.sort_values('word')
        result[word] = temp
    return result


def detect_lang_croup(df:pd.Series) -> None:
    from langdetect import detect_langs
    temp = df.str.replace('[.,\/#!?$%\^&\*;:{}=\-_`~()"0-9]','',regex = True)
    temp = temp.str.replace('[\u200f]','',regex = True)
    temp = temp.replace(r'^\s*$', np.nan, regex=True) # remove cell contain only spaces
    temp = temp.replace('[]',np.nan) #remove []
    temp = temp[temp.notnull()]
    result = temp.apply(lambda x: detect_langs(x)[0].lang)
    result = result.value_counts()
    result = result.to_frame().reset_index()
    return result
    fig = px.pie(result, values='line', names='index', title='Distribution of languages in the text')
    return fig


def add_entity(x):
    if x._.confidence_score > 0.75:
        return [x.text, x.label_]
    else:
        return ['','']
def entity_fun(x):
    lst_text = []
    lst_label = []
    rows = []
    doc = nlp(x)
    for entity in doc.ents:
        value = add_entity(entity)
        # lst_text.append(value[0])
        # lst_label.append(value[1])
        rows.append(pd.Series({'text' : value[0], 'label' :value[1]}))
    return tuple(rows)

def entity(df:pd.Series,plot = True):
    from tqdm import tqdm
    import spacy
    global nlp
    nlp = spacy.load("he_ner_news_trf")
    tqdm.pandas()
    data_entity = df.progress_apply(lambda x: entity_fun(x))
    data = pd.DataFrame(data_entity.values, columns= ['line'])
    data_x = data.explode('line')
    data_x = data_x[~(data_x.line.isna())]
    data_x = data_x.reset_index(drop=True)
    temp = pd.DataFrame(data_x['line'].tolist(), columns= ['text','label'])
    temp = temp[temp['text'].notnull()]
    return temp
    if plot:
        import plotly.express as px
        data_canada = temp[temp['label'] == 'PERS']['text'].value_counts().reset_index().head(10)
        fig1 = px.bar(data_canada, x='index', y='text',title = 'entity: persons')

        data_canada = temp['text'].value_counts().reset_index().head(10)
        fig2 = px.bar(data_canada, x='index', y='text',title = 'entity: all')

        data_canada = temp['label'].value_counts().reset_index()
        data_canada.columns = ['lable','index']
        fig3 = px.bar(data_canada, x='lable', y='index',title = 'entity: Distribution')
        return (fig1,fig2,fig3,temp)
    else:
        return temp

def translate_helper(x : str) -> str:
    """
    get text, translate it to english and perform pos
    """
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    from transformers import MarianMTModel, MarianTokenizer
    from flair.data import Sentence
    from flair.models import SequenceTagger
    import re
    tagger = SequenceTagger.load("flair/pos-english")
    result = []
    x = re.split('\.|/',x) # We should also provide \ as a separator for poems
    def split_every_30_char(string,n):
        words = string.split()
        return [' '.join(words[i: i + n]) for i in range(0, len(words), n)]
#     print([True if len(index) > 30 else False for index in x])
    N = 4
    x = [split_every_30_char(index,N) if len(index) > N else index for index in x]
    def flatten(t):
        return [item for sublist in t for item in sublist]
    x = flatten(x)
    
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    tokenizer = AutoTokenizer.from_pretrained("tiedeman/opus-mt-he-en")
    model = AutoModelForSeq2SeqLM.from_pretrained("tiedeman/opus-mt-he-en")
    
#     model_name = "tiedeman/opus-mt-he-en"
#     tokenizer = MarianTokenizer.from_pretrained(model_name)
    for item in x:
        if isinstance(item,str):
#             item = GoogleTranslator(source='auto', target='en').translate(item)
            
            translated = model.generate(**tokenizer(item, return_tensors="pt", padding=True))
            item = ' '.join([tokenizer.decode(t, skip_special_tokens=True) for t in translated])
#             return item
            sentence = Sentence(item)
            tagger.predict(sentence)
            item = [i.value for i in sentence.get_labels('pos')]
            result = result + item
    return result


def pos(df:pd.Series):
    """
    return plot of pos in the text
    """
    from flair.data import Sentence
    from flair.models import SequenceTagger
    tagger = SequenceTagger.load("flair/pos-english")
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    from collections import Counter
    from tqdm import tqdm

    tokenizer = AutoTokenizer.from_pretrained("tiedeman/opus-mt-he-en")
    model = AutoModelForSeq2SeqLM.from_pretrained("tiedeman/opus-mt-he-en")
    tqdm.pandas()
    translated = df.progress_apply(lambda x: translate_helper(x))
    temp = translated.apply(pd.Series).stack().value_counts().to_frame().reset_index()
    temp.columns = ['pos','sum']
    return temp
    fig = px.bar(temp, y='pos', x="sum", title="Parts Of Speech")
    return fig

def Zipf_law(df:pd.Series):
    from sklearn.feature_extraction.text import CountVectorizer
    df = df.replace('[^\u0590-\u05fe a-zA-Z]',' ',regex=True)
    # Preparations
    vec = CountVectorizer(ngram_range=(1, 1)).fit(df)
    bag_of_words = vec.transform(df)
    sum_words = bag_of_words.sum(axis=0)

    temp = pd.DataFrame(zip(vec.get_feature_names(),bag_of_words.sum(axis=0).tolist()[0]))
    temp.columns = ['words','freq']
    temp = temp.sort_values('freq',ascending= False)
    temp  = temp.reset_index(drop = True).reset_index(drop = False)
    temp = temp.rename(columns = {'index':'rank'})
    temp['rank'] = temp['rank'] + 1
    temp['Zipf'] = (temp.loc[0,'freq']/temp['rank'])

    temp['freq'] = temp['freq']/temp['freq'].sum()
    temp['Zipf'] = temp['Zipf']/temp['Zipf'].sum()
    
    return temp
    
    fig = px.line(temp,y=['freq','Zipf'], x='rank')
    fig.update_layout(xaxis_range=[0,30])
    return fig

def Connecting_words(x):
    x = x.replace(".", " ")
    x = ' '.join(x.split())
    x = x.split(' ')
    return [len(i) for i in x]

def len_word(df:pd.Series):
    temp = df.apply(lambda x: Connecting_words(x))
    import statistics
    temp = sum(temp.tolist(), [])
    return(statistics.mean(temp))

In [ ]:
# save the data

def database_comparison(df,df_altr,fun,titl):
    if df_altr.shape[0] > 0:
        dat = pd.DataFrame({'shape' : [fun(df['line']), fun(df_altr['line'])],'name' : ['main df','alternative df']})
        fig = px.pie(dat, values='shape', names='name',title = titl)
        fig.update_traces(hoverinfo='label+percent', textinfo='value')
        return fig
    else:
        return fun(df)

def save_pickle(obj,file_name):
    import pickle
    with open(file_name, 'wb') as handle:
                pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
def save_function(df : pd.DataFrame,df_altr : pd.DataFrame,dirc:str):
    df = df[df['line'].notnull()]
    df['line'] = df['line'].astype('str')
    
    #len_word
    temp = len_word(df['line'])
    save_pickle(temp,'len words.pickle')
    #Zipf_law
    temp = Zipf_law(df['line'])
    save_pickle(temp,'Zipf law.pickle')
    #number lines
    temp = number_lines(df['line'])
    save_pickle(temp,'number lines.pickle')
    #Character distribution
    temp = char_freq(df['line'],True)
    save_pickle(temp,'Character distribution.pickle')
    #ngram
    for x in range(1,6):
        temp = ngram(df['line'],x)
        save_pickle(temp,'ngram ' + str(x) + '.pickle')
    #top 5 words and another stat
    temp = ngram(df['line'],1)
    save_pickle(temp,'another stat.pickle')
    #identity duplicate line
    temp = identity_duplicate_line(df['line'])
    save_pickle(temp,'identity duplicate line.pickle')
    #topic model
    temp = topic_model(df['line'])
    save_pickle(temp,'topic model.pickle')
    #co occurrence
    temp = co_occurrence(df['line'],dirc)
    save_pickle(temp,'co occurrence.pickle')
    #number word in line
    temp = number_word_in_line(df['line'])
    save_pickle(temp,'number word in line.pickle')
    #Language recognition in corpus
    temp = detect_lang_croup(df['line'])
    save_pickle(temp,'Language recognition in corpus.pickle')

    
def set_cd(dirc):       
    create_folder(path_main + dirc)
    os.chdir(path_main + dirc)
    
def load_database():
#     lst = ['number lines','number unique words','Character distribution','ngram','another stat','identity duplicate line','topic model','co occurrence','number word in line','Language recognition in corpus','pos','entity']
    n = 100
    
    #ben yehuda
    print('ben yehuda')
    df = pd.read_csv(r'C:\corpus\ben yehuda\df.csv')
    df = df.sample(n)
    df_altr = pd.read_csv(r'C:\corpus\gutenberg\df.csv')
    df_altr = df_altr[['0']]
    df_altr.columns = ['line']
    set_cd(r'\ben yehuda')
    save_function(df,df_altr,'ben yehuda')
    
    ###think IL
    print('think IL')
    df = pd.read_csv(r'C:\corpus\think il\df think IL token.tsv',sep = '\t')
    df = df.sample(n)
    df = df.rename(columns={'text':'line','auther':'author'})
    df_altr = pd.DataFrame()
    set_cd(r'\think IL')
    save_function(df,df_altr,'think IL')
    

    #paraShoot
    print('paraShoot')
    df = pd.read_csv('C:\corpus\ParaShoot\df.csv',sep = '\t')
    df = df.rename(columns = {'question':'line'})
    df = df.sample(n)
    df_altr = pd.DataFrame()
    set_cd(r'\paraShoot')
    save_function(df,df_altr,'paraShoot')
    
    #Hebrew-Sentiment
    print('Hebrew-Sentiment')
    df = pd.read_csv('C:\corpus\Hebrew-Sentiment-Data\dev.tsv',sep = '\t')
    df = df.rename(columns = {'comment':'line'})
    df = df.sample(n)
    df_altr = pd.DataFrame()
    set_cd(r'\Hebrew-Sentiment')
    save_function(df,df_altr,'Hebrew-Sentiment')
    
    #HeBERT Emotion Recognition        
    print('HeBERT')
    df = pd.read_csv(r'C:\corpus\HeBERT Emotion Recognition\test_insample_tagged_raw_to_publish.csv')
    df = df.rename(columns = {'talkbacks':'line'})
    df = df.sample(n)
    df_altr = pd.DataFrame()
    set_cd(r'\HeBERT Emotion Recognition')
    save_function(df,df_altr,'HeBERT Emotion Recognition')

    #wikipedia
    print('wikipedia')
    df = pd.read_csv(r'C:\corpus\subset wiki\ he df.csv',sep = '\t')
    df = df.rename(columns = {'content':'line'})
    df = df.head(n)
    df_altr = pd.read_csv(r'C:\corpus\subset wiki\ en df.csv',sep = '\t')
    df_altr = df_altr.rename(columns = {'content':'line'})
    set_cd(r'\wikipedia')
    save_function(df,df_altr,'wikipedia')
    
    #mila
    global file
    for root, dirs, files in os.walk(r'C:\corpus\mila'):
        for file in files:
            if file.endswith('.csv'):
                print(file)
                df = pd.read_csv('C:\\corpus\\mila\\'+ file,sep = '\t')
                df = df.rename(columns = {'text':'line'})
                set_cd('\\' + file.split('.')[0])
#                 df = df.sample(n)
                if (file == 'HaKnesset (mila).csv') or (file == 'Tapuz People Forum (mila).csv'):
                    df = df.sample(5)
#                     df['line'] = df['line'].sample(10)#.str.split().str[:500].str.join(' ')
                df_altr = pd.DataFrame()
                save_function(df,df_altr,file.split('.')[0])
load_database()

ben yehuda
think IL
paraShoot
Hebrew-Sentiment
HeBERT
wikipedia
Arutz 7 (mila).csv
Arutz 7 df.csv
Doctors (mila).csv
Doctors df.csv
Foodpage Corpus (mila).csv
Foodpage Corpus df.csv
haaretz (mila).csv
haaretz df.csv
HaKnesset (mila).csv
HaKnesset df.csv


MemoryError: Unable to allocate 190. GiB for an array with shape (159659, 159659) and data type int64

# Dataset information

In [ ]:
# from plotly.subplots import make_subplots

# def load_pickle(dirc:str,file: str):
#     a = path_main + '\\' + dirc +'\\' +  file
#     with open(a, 'rb') as handle:
#         return pickle.load(handle)


# def info_dataset(dirc,word_co,ngram):
#     if (dirc == 'Select'):
#         pass
#     else:
#         titles = ('','Language recognition in corpus','ngram','co occurrence','entity all','entity text','entity person','Character distribution','pos','topic','Zipf law')
#         fig = make_subplots(
#             rows=6, cols=3,
#             specs=[[{"type":"table","colspan":3,"rowspan":1},None,None],
#                   [{"type": "pie"},{"type": "bar"},{"type": "bar"}],
#                   [{"type": "bar"},{"type": "bar"},{"type": "bar"}],
#                   [{"type": "bar","rowspan":2},{"type": "bar",'rowspan':2},{"type": "table",'rowspan':2}],
#                   [None,None,None],
#                   [{"type": "Scatter","colspan":2},None,None]],
#             subplot_titles = titles
#                 )

#         #stat
#         stat = load_pickle(dirc,'another stat.pickle')
#         stat.pop('top unqiue words',None)
# #         stat.pop('tail unique words',None)
#         stat['Average len words'] = load_pickle(dirc,'len words.pickle')
#         stat['number lines'] = load_pickle(dirc,'number lines.pickle')
#         stat['Percent appeared once'] = np.round(stat['Percent appeared once'],2)
#         stat['type-token ratio'] = np.round(stat['type-token ratio'],2)
#         stat['Average len words'] = np.round(stat['Average len words'],2)
#         stat['gini'] = np.round(stat['gini'],2)
        
#         #rename
#         stat['Number Unique Words'] = stat.pop('shape unique words')
#         stat['Number Appeares Once'] = stat.pop('shape appeared once')
#         stat['Percent Appeared Once'] = stat.pop('Percent appeared once')
#         stat['Number Of Words'] = stat.pop('Number Of Words')
#         stat['Type-Token Ratio'] = stat.pop('type-token ratio')
#         stat['Gini'] = stat.pop('gini')
#         stat['Average len words'] = stat.pop('Average len words')
#         stat['Number Lines'] = stat.pop('number lines')
        

        
#         stat_plot = go.Table(header=dict(values=list(stat.keys())),
#                          cells=dict(values=list(stat.values())))
#         fig.add_trace(stat_plot,row=1, col=1)

#         #Language recognition in corpus
#         temp = load_pickle(dirc,'Language recognition in corpus.pickle')
#         plot = go.Pie(labels=temp['index'], values=temp['line'])
#         fig.add_trace(plot,row=2, col=1)

#         #ngram
#         a = path_main + '\\' + dirc + '\\ngram ' + str(ngram) + '.pickle'
#         with open(a, 'rb') as handle:
#             temp = pickle.load(handle)['top unqiue words'].sort_values('frequency')
#             ngram_plot = go.Bar(y=temp['word'],
#                     x=temp['frequency'],
#                         orientation='h')
#         fig.add_trace(ngram_plot,row=2, col=2)

#         #co occurrence
#         df = load_pickle(dirc,'co occurrence.pickle')
#         df = df[word_co].nlargest(10).to_frame().reset_index()
#         df.columns = ['number','word']
#         df = df.sort_values('word')
#         co_plot = go.Bar(x=df['word'],
#                                      y=df['number'],
#                                     orientation='h')
#         fig.add_trace(co_plot,row=2, col=3)

#         #entity
#         temp = load_pickle(dirc,'entity.pickle')
#         temp = temp[temp['label'] != '']
#         entity_all = temp['label'].value_counts().reset_index()
#         entity_all.columns = ['lable','index']
#         entity_all = entity_all.sort_values('index')
#         fig.add_trace(go.Bar(y=entity_all['lable'],
#                                          x=entity_all['index'],
#                                         orientation='h'),
#                                   row=3, col=1)


#         entity_text = temp['text'].value_counts().reset_index().head(10)
#         entity_text = entity_text.sort_values('text')
#         fig.add_trace(go.Bar(y=entity_text['index'],
#                                          x=entity_text['text'],
#                                         orientation='h'),
#                                   row=3, col=2)


#         entity_pers = temp[temp['label'] == 'PERS']['text'].value_counts().reset_index().head(10)
#         entity_pers = entity_pers.sort_values('text')
#         entity_pers = go.Bar(y=entity_pers['index'],
#                     x=entity_pers['text'],
#                     orientation='h')

#         fig.add_trace(entity_pers,row=3, col=3)


#         #Character distribution
#         chr_freq = load_pickle(dirc,'Character distribution.pickle')
#         chr_freq = pd.DataFrame(chr_freq.items(),columns = ['char','freq'])
#         chr_freq = go.Bar(y=chr_freq['char'],x=chr_freq['freq'],orientation='h')
#         fig.add_trace(chr_freq,row=4, col=1)

#         #pos
#         pos_plot = load_pickle(dirc,'pos.pickle').sort_values('sum')
#         pos_plot = go.Bar(y=pos_plot['pos'],x=pos_plot['sum'],orientation='h')
#         fig.add_trace(pos_plot,row=4, col=2)

#         #topic
#         temp = load_pickle(dirc,'topic model.pickle')
#         temp = go.Table(header=dict(values=['Topic 1','Topic 2','Topic 3','Topic 4']),
#                          cells=dict(values=temp))
#         fig.add_trace(temp,row=4, col=3)

#         #Zipf law
#         temp = load_pickle(dirc,'Zipf law.pickle').head(20)
#         fig.add_trace(go.Scatter(
#                                 x = temp['rank'],
#                                 y = temp['freq'],
#                                 name = 'freq',
#                                 text = temp['words']),
#                      row=6, col=1)
    
#         fig.add_trace(go.Scatter(
#                                 x = temp['rank'],
#                                 y = temp['Zipf'],
#                                 name = 'Zipf Law'),
#                      row=6, col=1)
        
#         fig.update_layout(height=1200,width = 900, showlegend=False)
#         fig.show()



        
# def set_plotType(_):
#     """
#     change the lable on button co_widget base on change in data (data_widget)
#     """
#     if data_widget.value == 'Select':
#         pass
#     else:
#         dirc = data_widget.value
#         lst_co = load_pickle(dirc,'ngram 1.pickle')['top unqiue words']['word'].head().to_list()
#         co_widget.options = lst_co
# #         info_dataset(dirc,lst_co[0],1)
        
# #data
# lst_data = ['Select','ben yehuda','think IL','paraShoot','Hebrew-Sentiment','HeBERT Emotion Recognition','wikipedia'
#             ,'Arutz 7 df','Doctors df','Foodpage Corpus df','haaretz df','HaKnesset df','Hebrew Dotted Text df','Infomed df','Learning Man df','Nature of Healing df','Spoken Israeli Hebrew df','Sport5 Corpus df','Tapuz People Forum df','TheMarker df','To Be Healthy df','Walla Food Corpus df']
# data_widget = widgets.Dropdown(options = lst_data,description='Chose data',value = 'Select')

# #ngram
# ngram_slider = widgets.IntSlider(min=1,max=5,step=1,value=1)

# # co-occurrence
# co_widget = widgets.Dropdown(
#         options=['Select'],
#         description='select word'
#     )

# data_widget.observe(set_plotType)


# # def helppp(_):
# #     info_dataset(data_widget.value,co_widget.value, ngram_slider.value)
# # ngram_slider.observe(helppp)

# # interact(info_dataset,dirc = data_widget, word_co = co_widget, ngram = ngram_slider)

# temp = widgets.interactive(info_dataset,dirc = data_widget, word_co = co_widget, ngram = ngram_slider)
# display(temp)
# # def a(dirc):
# #     info_dataset(dirc,co_widget.value,ngram_slider.value)
# # def b(word_co):
# #     info_dataset(data_widget.value,word_co,ngram_slider.value)
# # def c(ngram):
# #     info_dataset(data_widget.value,co_widget.value,ngram)
# # a = widgets.interactive(a,dirc = data_widget)
# # b = widgets.interactive(b,word_co = co_widget)
# # c = widgets.interactive(c,ngram = ngram_slider)



# # grid = widgets.GridspecLayout(1,3)
# # grid[0, 0] = a
# # grid[0, 1] = b
# # grid[0, 2] = c
# # # grid[1, 2] = info_dataset

# # display(grid)



In [ ]:
# from plotly.subplots import make_subplots

# def load_pickle(dirc:str,file: str):
#     a = path_main + '\\' + dirc +'\\' +  file
#     with open(a, 'rb') as handle:
#         return pickle.load(handle)



# dirc,ngram = 'ben yehuda',1
# word_co = load_pickle(dirc,'ngram 1.pickle')['top unqiue words']['word'].head().to_list()[0]

# #stat
# stat = load_pickle(dirc,'another stat.pickle')
# stat.pop('top unqiue words',None)
# #         stat.pop('tail unique words',None)
# stat['Average len words'] = load_pickle(dirc,'len words.pickle')
# stat['number lines'] = load_pickle(dirc,'number lines.pickle')
# stat['Percent appeared once'] = np.round(stat['Percent appeared once'],2)
# stat['type-token ratio'] = np.round(stat['type-token ratio'],2)
# stat['Average len words'] = np.round(stat['Average len words'],2)
# stat['gini'] = np.round(stat['gini'],2)

# #rename
# stat['Number Unique Words'] = stat.pop('shape unique words')
# stat['Number Appeares Once'] = stat.pop('shape appeared once')
# stat['Percent Appeared Once'] = stat.pop('Percent appeared once')
# stat['Number Of Words'] = stat.pop('Number Of Words')
# stat['Type-Token Ratio'] = stat.pop('type-token ratio')
# stat['Gini'] = stat.pop('gini')
# stat['Average len words'] = stat.pop('Average len words')
# stat['Number Lines'] = stat.pop('number lines') 
# stat_plot = go.Table(header=dict(values=list(stat.keys())),
#                  cells=dict(values=list(stat.values())))
# stat_plot = go.FigureWidget(stat_plot)
# stat_plot.layout.height = 270
# stat_plot.layout.title.text = 'Info Card'
# stat_plot.layout.title.x = 0.5
# stat_plot.layout.title.y = 0.85

# #Language recognition in corpus
# temp = load_pickle(dirc,'Language recognition in corpus.pickle')
# lang_plot = go.Pie(labels=temp['index'], values=temp['line'])
# lang_plot = go.FigureWidget(lang_plot)
# lang_plot.layout.title.text = 'Languages'
# lang_plot.layout.title.x = 0.5
# lang_plot.layout.title.y = 0.85

# #co occurrence
# df = load_pickle(dirc,'co occurrence.pickle')
# word_co = list(df.keys())[0]
# df = df[word_co]
# co_plot = go.Bar(x=df['word'],
#                              y=df['number'],
#                             orientation='h')
# co_plot = go.FigureWidget(co_plot)       
# co_plot.layout.title.text = 'Co Occurrence'
# co_plot.layout.title.x = 0.5
# co_plot.layout.title.y = 0.85

# # #entity
# # temp = load_pickle(dirc,'entity.pickle')
# # temp = temp[temp['label'] != '']
# # entity_all = temp['label'].value_counts().reset_index()
# # entity_all.columns = ['lable','index']
# # entity_all = entity_all.sort_values('index')
# # entity_all_plt = go.Bar(y=entity_all['lable'],
# #                                  x=entity_all['index'],
# #                                 orientation='h')
# # entity_all_plt = go.FigureWidget(entity_all_plt)        


# # entity_text = temp['text'].value_counts().reset_index().head(10)
# # entity_text = entity_text.sort_values('text')
# # entity_text_plt = go.Bar(y=entity_text['index'],
# #                                  x=entity_text['text'],
# #                                 orientation='h')
# # entity_text_plt = go.FigureWidget(entity_text_plt)        

# # entity_pers = temp[temp['label'] == 'PERS']['text'].value_counts().reset_index().head(10)
# # entity_pers = entity_pers.sort_values('text')
# # entity_pers_plt = go.Bar(y=entity_pers['index'],
# #             x=entity_pers['text'],
# #             orientation='h')
# # entity_pers_plt = go.FigureWidget(entity_pers_plt)        

# #Character distribution
# chr_freq = load_pickle(dirc,'Character distribution.pickle')
# chr_freq = pd.DataFrame(chr_freq.items(),columns = ['char','freq'])
# chr_freq_plt = go.Bar(y=chr_freq['char'],x=chr_freq['freq'],orientation='h')
# chr_freq_plt = go.FigureWidget(chr_freq_plt)
# chr_freq_plt.layout.title.text = 'Character Distribution'
# chr_freq_plt.layout.title.x = 0.5
# chr_freq_plt.layout.title.y = 0.85

# # #pos
# # pos_plot = load_pickle(dirc,'pos.pickle').sort_values('sum')
# # pos_plot_plt = go.Bar(y=pos_plot['pos'],x=pos_plot['sum'],orientation='h')
# # pos_plot_plt = go.FigureWidget(pos_plot_plt)        


# #Zipf law
# temp = load_pickle(dirc,'Zipf law.pickle').head(20)
# Zipf_plt = go.Figure()
# Zipf_plt.add_trace(go.Scatter(
#                         x = temp['rank'],
#                         y = temp['freq'],
#                         name = 'freq',
#                         text = temp['words']),
# )

# Zipf_plt.add_trace(go.Scatter(
#                         x = temp['rank'],
#                         y = temp['Zipf'],
#                         name = 'Zipf Law'),
# )
# Zipf_plt = go.FigureWidget(Zipf_plt)    
# Zipf_plt.layout.title.text = 'Zipf Law'
# Zipf_plt.layout.title.x = 0.5
# Zipf_plt.layout.title.y = 0.85

# #topic
# temp = load_pickle(dirc,'topic model.pickle')
# topic_plt = go.Table(header=dict(values=['Topic 1','Topic 2','Topic 3','Topic 4']),
#                  cells=dict(values=temp))
# topic_plt = go.FigureWidget(topic_plt) 
# topic_plt.layout.title.text = 'Topics'
# topic_plt.layout.title.x = 0.5
# topic_plt.layout.title.y = 0.85


# #ngram
# a = path_main + '\\' + dirc + '\\ngram ' + str(ngram) + '.pickle'
# with open(a, 'rb') as handle:
#     temp = pickle.load(handle)['top unqiue words'].sort_values('frequency')
#     ngram_plot = go.Bar(y=temp['word'],
#             x=temp['frequency'],
#                 orientation='h')
# ngram_plot = go.FigureWidget(ngram_plot)    
# ngram_plot.layout.title.text = 'ngram'
# ngram_plot.layout.title.x = 0.5
# ngram_plot.layout.title.y = 0.85


# def set_plotType(_):
#     """
#     change the lable on button co_widget base on change in data (data_widget)
#     """
#     if data_widget.value == 'Select':
#         pass
#     else:
#         df = load_pickle(dirc,'co occurrence.pickle')
#         word_co = list(df.keys())
# #         dirc = data_widget.value
# #         lst_co = load_pickle(dirc,'ngram 1.pickle')['top unqiue words']['word'].head().to_list()
#         co_widget.options = word_co
        
# #data
# lst_data = ['ben yehuda','think IL','paraShoot','Hebrew-Sentiment','HeBERT Emotion Recognition','wikipedia'
#             ,'Arutz 7 (mila)','Doctors (mila)','Foodpage Corpus (mila)','haaretz (mila)','HaKnesset (mila)','Hebrew Dotted Text (mila)','Infomed (mila)','Learning Man (mila)','Nature of Healing (mila)','Spoken Israeli Hebrew (mila)','Sport5 Corpus (mila)','Tapuz People Forum (mila)','TheMarker (mila)','To Be Healthy (mila)','Walla Food Corpus (mila)']
# data_widget = widgets.Dropdown(
#     options = lst_data,
#     description='Chose data',
#     value = 'think IL'
# )

# #ngram
# ngram_slider = widgets.IntSlider(
#     description = 'n-gram',
#     min=1,
#     max=5,
#     step=1,
#     value=1
# )

# # co-occurrence
# word_list = load_pickle(dirc,'ngram 1.pickle')['top unqiue words']['word'].head().to_list()
# co_widget = widgets.Dropdown(
#     options=word_list,
#     description='select word'
#     )

# # data_widget.observe(set_plotType)


# fig = go.FigureWidget()

# def response_data(change):
#     dirc = data_widget.value
#     ngram_int = ngram_slider.value
#     word_co = co_widget.value
    
#     df_jfjf = load_pickle(dirc,'co occurrence.pickle')
#     co_widget.options = list(df_jfjf.keys())
#     response_ngram(' ')
#     if word_co == 'Select':
#         pass
#     else:
#         with fig.batch_update():

#             stat = load_pickle(dirc,'another stat.pickle')
#             stat.pop('top unqiue words',None)
#             stat['Average len words'] = load_pickle(dirc,'len words.pickle')
#             stat['number lines'] = load_pickle(dirc,'number lines.pickle')
#             stat['Percent appeared once'] = np.round(stat['Percent appeared once'],2)
#             stat['type-token ratio'] = np.round(stat['type-token ratio'],2)
#             stat['Average len words'] = np.round(stat['Average len words'],2)
#             stat['gini'] = np.round(stat['gini'],2)
            
#             #rename
#             stat['Number Unique Words'] = stat.pop('shape unique words')
#             stat['Number Appeares Once'] = stat.pop('shape appeared once')
#             stat['Percent Appeared Once'] = stat.pop('Percent appeared once')
#             stat['Number Of Words'] = stat.pop('Number Of Words')
#             stat['Type-Token Ratio'] = stat.pop('type-token ratio')
#             stat['Gini'] = stat.pop('gini')
#             stat['Average len words'] = stat.pop('Average len words')
#             stat['Number Lines'] = stat.pop('number lines') 
            
#             stat_plot.data[0].cells.values = list(stat.values())
            
#             #Language recognition in corpus
#             temp = load_pickle(dirc,'Language recognition in corpus.pickle')
#             lang_plot.data[0].labels = temp['index']
#             lang_plot.data[0].values = temp['line']

# #             #entity
# #             temp = load_pickle(dirc,'entity.pickle')
# #             temp = temp[temp['label'] != '']
# #             entity_all = temp['label'].value_counts().reset_index()
# #             entity_all.columns = ['lable','index']
# #             entity_all = entity_all.sort_values('index')
# #             entity_all_plt.data[0].x = entity_all['index']
# #             entity_all_plt.data[0].y = entity_all['lable']
     

# #             entity_text = temp['text'].value_counts().reset_index().head(10)
# #             entity_text = entity_text.sort_values('text')
# #             entity_text_plt.data[0].x = entity_text['text']
# #             entity_text_plt.data[0].y = entity_text['index']

# #             entity_pers = temp[temp['label'] == 'PERS']['text'].value_counts().reset_index().head(10)
# #             entity_pers = entity_pers.sort_values('text')
# #             entity_pers_plt.data[0].x = entity_pers['text']
# #             entity_pers_plt.data[0].y = entity_pers['index']   
            
#             #Character distribution
#             chr_freq = load_pickle(dirc,'Character distribution.pickle')
#             chr_freq = pd.DataFrame(chr_freq.items(),columns = ['char','freq'])
#             chr_freq_plt.data[0].x = chr_freq['freq']
#             chr_freq_plt.data[0].y = chr_freq['char']

        
# #             #pos
# #             pos_plot = load_pickle(dirc,'pos.pickle').sort_values('sum')
# #             pos_plot_plt.data[0].x = pos_plot['sum']
# #             pos_plot_plt.data[0].y = pos_plot['pos']
            
#             #topic model
#             temp = load_pickle(dirc,'topic model.pickle')
#             topic_plt.data[0].cells = dict(values=temp)
            
#             #Zipf law
#             temp = load_pickle(dirc,'Zipf law.pickle').head(20)
#             Zipf_plt.data[0].x = temp['rank']
#             Zipf_plt.data[0].y = temp['freq']
#             Zipf_plt.data[0].text = temp['words']

# def response_ngram(change):
#     dirc = data_widget.value
#     ngram_int = ngram_slider.value
#     word_co = co_widget.value
#     with fig.batch_update():
#         #ngram
#             a = path_main + '\\' + dirc + '\\ngram ' + str(ngram_int) + '.pickle'
#             with open(a, 'rb') as handle:
#                 temp = pickle.load(handle)['top unqiue words'].sort_values('frequency')
#                 ngram_plot.data[0].x = temp['frequency']
#                 ngram_plot.data[0].y = temp['word']
                
# def response_co(change):
#     # co-occurrence
#     dirc = data_widget.value
#     ngram_int = ngram_slider.value
#     word_co = co_widget.value
#     with fig.batch_update():
#             df = load_pickle(dirc,'co occurrence.pickle')
#             df = df[word_co]
#             co_plot.data[0].x = df['word']
#             co_plot.data[0].y = df['number']
                
# ngram_slider.observe(response_ngram, names="value")
# co_widget.observe(response_co, names="value")
# data_widget.observe(response_data, names="value")


# plots_1 = widgets.HBox([lang_plot,ngram_plot,co_plot])
# # plots_2 = widgets.HBox([entity_all_plt,entity_text_plt,entity_pers_plt])
# plots_3 = widgets.HBox([chr_freq_plt,topic_plt]) #pos_plot_plt
# plots_4 = widgets.HBox([Zipf_plt])


# container  = widgets.HBox([data_widget,ngram_slider,co_widget])

# widgets.VBox([stat_plot,
#               container,
#               plots_1,
# #               plots_2,
#               plots_3,
#               plots_4
# ])






In [ ]:
from plotly.subplots import make_subplots

def load_pickle(dirc:str,file: str):
    a = path_main + '\\' + dirc +'\\' +  file
    with open(a, 'rb') as handle:
        return pickle.load(handle)


dirc1,ngram = 'ben yehuda',1
dirc2 = 'haaretz (mila)'
word_co = load_pickle(dirc1,'ngram 1.pickle')['top unqiue words']['word'].head().to_list()[0]
word_co2 = load_pickle(dirc2,'ngram 1.pickle')['top unqiue words']['word'].head().to_list()[0]

#Character distribution
chr_freq = load_pickle(dirc1,'Character distribution.pickle')
chr_freq = pd.DataFrame(chr_freq.items(),columns = ['char','freq'])
chr_freq['freq'] = chr_freq['freq']/chr_freq['freq'].sum()
chr_freq_plt = go.Bar(y=chr_freq['char'],x=chr_freq['freq'],orientation='h')

chr_freq2 = load_pickle(dirc2,'Character distribution.pickle')
chr_freq2 = pd.DataFrame(chr_freq2.items(),columns = ['char','freq'])
chr_freq2['freq'] = chr_freq2['freq']/chr_freq2['freq'].sum()
chr_freq_plt2 = go.Bar(y=chr_freq2['char'],x=chr_freq2['freq'],orientation='h')

chr_freq_plt = go.Figure(data=[chr_freq_plt,chr_freq_plt2])
chr_freq_plt = go.FigureWidget(chr_freq_plt)
chr_freq_plt.layout.title.text = 'Character Distribution'
chr_freq_plt.layout.title.x = 0.5
chr_freq_plt.layout.title.y = 0.85

#ngram
#main
a = path_main + '\\' + dirc1 + '\\ngram ' + str(ngram) + '.pickle'
with open(a, 'rb') as handle:
    temp1 = pickle.load(handle)['top unqiue words'].sort_values('frequency')
    temp1['frequency'] = temp1['frequency']/temp1['frequency'].sum()
    temp1 = temp1.sort_values('frequency',ascending = False).head(10).sort_values('frequency')
    ngram_plot1 = go.Bar(name = dirc1,
            y=temp1['word'],
            x=temp1['frequency'],
                orientation='h')
#secondary
a = path_main + '\\' + dirc2 + '\\ngram ' + str(ngram) + '.pickle'
with open(a, 'rb') as handle:
    temp2 = pickle.load(handle)['top unqiue words'].sort_values('frequency')
    temp2['frequency'] = temp2['frequency']/temp2['frequency'].sum()
    temp2 = temp2[temp2['word'].isin(temp1['word'])]
    ngram_plot2 = go.Bar(name = dirc2,
            y=temp2['word'],
            x=temp2['frequency'],
                orientation='h')        
        

ngram_plot = go.Figure(data=[ngram_plot1,ngram_plot2])
ngram_plot = go.FigureWidget(ngram_plot)
ngram_plot.layout.title.text = 'ngram'
ngram_plot.layout.title.x = 0.5
ngram_plot.layout.title.y = 0.85

#stat
def stat_fun(dirc):
    stat = load_pickle(dirc,'another stat.pickle')
    stat.pop('top unqiue words',None)
    #         stat.pop('tail unique words',None)
    stat['Average len words'] = load_pickle(dirc,'len words.pickle')
    stat['number lines'] = load_pickle(dirc,'number lines.pickle')
    stat['Percent appeared once'] = np.round(stat['Percent appeared once'],2)
    stat['type-token ratio'] = np.round(stat['type-token ratio'],2)
    stat['Average len words'] = np.round(stat['Average len words'],2)
    stat['gini'] = np.round(stat['gini'],2)

    #rename
    stat['Number Unique Words'] = stat.pop('shape unique words')
    stat['Number Appeares Once'] = stat.pop('shape appeared once')
    stat['Percent Appeared Once'] = stat.pop('Percent appeared once')
    stat['Number Of Words'] = stat.pop('Number Of Words')
    stat['Type-Token Ratio'] = stat.pop('type-token ratio')
    stat['Gini'] = stat.pop('gini')
    stat['Average len words'] = stat.pop('Average len words')
    stat['Number Lines'] = stat.pop('number lines')
    return stat
stat = stat_fun(dirc1)
stat2 = stat_fun(dirc2)
lists_of_lists = [list(stat.values()),list(stat2.values())]
cells_stat = [x for x in zip(*lists_of_lists)]
cells_stat.insert(0,(dirc1,dirc2))
header_stat = list(stat.keys())
header_stat.insert(0,'DataSet')
stat_plot = go.Table(columnwidth = [100] + [60]*8,
                     header=dict(values=header_stat),
                 cells=dict(
                     values=cells_stat,
                    fill=dict(color=['lightgrey'] + ['white']*8
                             )))
stat_plot = go.FigureWidget(stat_plot)
stat_plot.layout.height = 350
stat_plot.layout.title.text = 'Info Card'
stat_plot.layout.title.x = 0.5
stat_plot.layout.title.y = 0.85

#Language recognition in corpus
temp = load_pickle(dirc1,'Language recognition in corpus.pickle')
lang_plot = go.Pie(labels=temp['index'], values=temp['line'])
lang_plot = go.FigureWidget(lang_plot)
lang_plot.layout.title.text = 'Languages'
lang_plot.layout.title.x = 0.5
lang_plot.layout.title.y = 0.85

#co occurrence
df = load_pickle(dirc1,'co occurrence.pickle')
word_co = list(df.keys())[0]
df = df[word_co]
co_plot = go.Bar(x=df['word'],
                             y=df['number'],
                            orientation='h')
co_plot = go.FigureWidget(co_plot)       
co_plot.layout.title.text = 'Co Occurrence'
co_plot.layout.title.x = 0.5
co_plot.layout.title.y = 0.85


#Zipf law
Zipf_plt = go.Figure()
#main
temp = load_pickle(dirc1,'Zipf law.pickle').head(20)
Zipf_plt.add_trace(go.Scatter(
                        x = temp['rank'],
                        y = temp['freq'],
                        name = 'freq ' + dirc1,
                        text = temp['words']),
)
#secondary
temp = load_pickle(dirc2,'Zipf law.pickle').head(20)
Zipf_plt.add_trace(go.Scatter(
                        x = temp['rank'],
                        y = temp['freq'],
                        name = 'freq ' + dirc2,
                        text = temp['words']),
)

Zipf_plt.add_trace(go.Scatter(
                        x = temp['rank'],
                        y = temp['Zipf'],
                        name = 'Zipf Law'),
)
Zipf_plt = go.FigureWidget(Zipf_plt)    
Zipf_plt.layout.title.text = 'Zipf Law'
Zipf_plt.layout.title.x = 0.5
Zipf_plt.layout.title.y = 0.85

#topic
temp = load_pickle(dirc1,'topic model.pickle')
topic_plt = go.Table(header=dict(values=['Topic 1','Topic 2','Topic 3','Topic 4']),
                 cells=dict(values=temp))
topic_plt = go.FigureWidget(topic_plt) 
topic_plt.layout.title.text = 'Topics'
topic_plt.layout.title.x = 0.5
topic_plt.layout.title.y = 0.85





def set_plotType(_):
    """
    change the lable on button co_widget base on change in data (data_widget)
    """
    if data_widget.value == 'Select':
        pass
    else:
        df = load_pickle(dirc,'co occurrence.pickle')
        word_co = list(df.keys())
#         dirc = data_widget.value
#         lst_co = load_pickle(dirc,'ngram 1.pickle')['top unqiue words']['word'].head().to_list()
        co_widget.options = word_co
        
#data
lst_data = ['ben yehuda','think IL','paraShoot','Hebrew-Sentiment','HeBERT Emotion Recognition','wikipedia'
            ,'Arutz 7 (mila)','Doctors (mila)','Foodpage Corpus (mila)','haaretz (mila)','HaKnesset (mila)',
            'Hebrew Dotted Text (mila)','Infomed (mila)','Learning Man (mila)','Nature of Healing (mila)',
            'Spoken Israeli Hebrew (mila)','Sport5 Corpus (mila)','Tapuz People Forum (mila)','TheMarker (mila)',
            'To Be Healthy (mila)','Walla Food Corpus (mila)']
data_widget = widgets.Dropdown(
    options = lst_data,
    description='main dataset',
    value = 'think IL'
)

data_widget2 = widgets.Dropdown(
    options = lst_data,
    description='secondary dataset',
    value = 'think IL'
)

#ngram
ngram_slider = widgets.IntSlider(
    description = 'n-gram',
    min=1,
    max=5,
    step=1,
    value=1
)
#stop words
stop_words_box = widgets.Checkbox(False, description='stop words')

# co-occurrence
word_list = load_pickle(dirc1,'ngram 1.pickle')['top unqiue words']['word'].head().to_list()
co_widget = widgets.Dropdown(
    options=word_list,
    description='select word'
    )

# data_widget.observe(set_plotType)


fig = go.FigureWidget()

def response_data(plt_number,change):
    dirc1 = data_widget.value
    dirc2 = data_widget2.value
    ngram_int = ngram_slider.value
    word_co = co_widget.value
    
    co_widget_lst = load_pickle(dirc1,'co occurrence.pickle')
    co_widget.options = list(co_widget_lst.keys())
    response_ngram(' ')
    if word_co == 'Select':
        pass
    else:
        with fig.batch_update():
            stat1 = stat_fun(dirc1)
            stat2 = stat_fun(dirc2)
            lists_of_lists = [list(stat1.values()),list(stat2.values())]
            cells_stat = [x for x in zip(*lists_of_lists)]
            cells_stat.insert(0,(dirc1,dirc2))

            stat_plot.data[0].cells.values = list(cells_stat)
            
            #Language recognition in corpus
            temp = load_pickle(dirc1,'Language recognition in corpus.pickle')
            lang_plot.data[0].labels = temp['index']
            lang_plot.data[0].values = temp['line']
            
            #Character distribution
            #main
            chr_freq1 = load_pickle(dirc1,'Character distribution.pickle')
            chr_freq1 = pd.DataFrame(chr_freq1.items(),columns = ['char','freq'])
            chr_freq1['freq'] = chr_freq1['freq']/chr_freq1['freq'].sum()
            chr_freq_plt.data[0].x = chr_freq1['freq']
            chr_freq_plt.data[0].y = chr_freq1['char']

            
            #secondary
            chr_freq2 = load_pickle(dirc2,'Character distribution.pickle')
            chr_freq2 = pd.DataFrame(chr_freq2.items(),columns = ['char','freq'])
            chr_freq2['freq'] = chr_freq2['freq']/chr_freq2['freq'].sum()
            chr_freq_plt.data[0].x = chr_freq2['freq']
            chr_freq_plt.data[0].y = chr_freq2['char']
            
            #topic model
            temp = load_pickle(dirc1,'topic model.pickle')
            topic_plt.data[0].cells = dict(values=temp)
            
            #Zipf law
            #main
            temp = load_pickle(dirc1,'Zipf law.pickle').head(20)
            Zipf_plt.data[0].x = temp['rank']
            Zipf_plt.data[0].y = temp['freq']
            Zipf_plt.data[0].text = temp['words']
            
            #secondary
            temp = load_pickle(dirc2,'Zipf law.pickle').head(20)
            Zipf_plt.data[1].x = temp['rank']
            Zipf_plt.data[1].y = temp['freq']
            Zipf_plt.data[1].text = temp['words']
        

def response_ngram(change):
    dirc1 = data_widget.value
    dirc2 = data_widget2.value
    ngram_int = ngram_slider.value
    word_co = co_widget.value
    with fig.batch_update():
        #ngram
        #main
        a = path_main + '\\' + dirc1 + '\\ngram ' + str(ngram_int) + '.pickle'
        with open(a, 'rb') as handle:
            temp1 = pickle.load(handle)['top unqiue words'].sort_values('frequency')
            temp1['frequency'] = temp1['frequency']/temp1['frequency'].sum()
            temp1 = temp1.sort_values('frequency',ascending = False).head(10).sort_values('frequency')   
            ngram_plot.data[0].x = temp1['frequency']
            ngram_plot.data[0].y = temp1['word']
            ngram_plot.data[0].name = dirc1
        
        #secondary
        a = path_main + '\\' + dirc2 + '\\ngram ' + str(ngram_int) + '.pickle'
        with open(a, 'rb') as handle:
            temp2 = pickle.load(handle)['top unqiue words'].sort_values('frequency')
            temp2['frequency'] = temp2['frequency']/temp2['frequency'].sum()
            temp2 = temp2[temp2['word'].isin(temp1['word'])]
            ngram_plot.data[1].x = temp2['frequency']
            ngram_plot.data[1].y = temp2['word']  
            ngram_plot.data[1].name = dirc2
                
def response_co(change):
    # co-occurrence
    dirc = data_widget.value
    ngram_int = ngram_slider.value
    word_co = co_widget.value
    with fig.batch_update():
            df = load_pickle(dirc,'co occurrence.pickle')
            df = df[word_co]
            co_plot.data[0].x = df['word']
            co_plot.data[0].y = df['number']
                
                
def response_data1(change):
    response_data(0,change)
        
def response_data2(change):
    response_data(1,change)
        
        
ngram_slider.observe(response_ngram, names="value")
co_widget.observe(response_co, names="value")
data_widget.observe(response_data1, names="value")
data_widget2.observe(response_data2, names="value")

plots_1 = widgets.HBox([lang_plot,ngram_plot,co_plot])
# plots_2 = widgets.HBox([entity_all_plt,entity_text_plt,entity_pers_plt])
plots_3 = widgets.HBox([chr_freq_plt,topic_plt]) #pos_plot_plt
plots_4 = widgets.HBox([Zipf_plt])


container1 = widgets.HBox([data_widget,data_widget2,stop_words_box])
container2 = widgets.HBox([ngram_slider,co_widget])
container = widgets.VBox([container1,container2])
widgets.VBox([stat_plot,
              container,
              plots_1,
#               plots_2,
              plots_3,
              plots_4
])







    'data': [{'cells': {'fill': {'color': [lightgrey, white, white, white, white…

In [ ]:
stat = stat_fun(dirc1)
stat2 = stat_fun(dirc2)
lists_of_lists = [list(stat.values()),list(stat2.values())]
cells_stat = [x for x in zip(*lists_of_lists)]
cells_stat.insert(0,(dirc1,dirc2))
header_stat = list(stat.keys())
header_stat.insert(0,'DataSet')

stat_plot = go.FigureWidget(stat_plot)
stat_plot.layout.height = 350
stat_plot.layout.title.text = 'Info Card'
stat_plot.layout.title.x = 0.5
stat_plot.layout.title.y = 0.85
go.Figure(stat_plot)

In [ ]:




ngram_plot

FigureWidget({
    'data': [{'name': 'hi',
              'orientation': 'h',
              'type': 'bar',
    …

# Focused look at a dataset

In [ ]:
def over_view(df: pd.DataFrame,col : str, n : int,plot_text : str,plot = 'none'):
    temp = df[col].value_counts().reset_index().sort_values(col,ascending=False).reset_index(drop = True)
    temp = temp.replace(r'\s+', ' ', regex=True)
    temp.loc[temp['index'] == ' ','index'] = 'לא ידוע'
    sum_all = temp[col].sum()
    temp = temp[(temp[col]/temp[col].sum()).cumsum() < 0.90]
    temp = temp.append({'index': 'others',col: sum_all - temp[col].sum()}, ignore_index=True)
    return temp
        
def remove_stop_word(df:pd.Series) -> pd.Series:
    stop = pd.read_csv(r'C:\corpus\heb_stopwords.txt', sep=" ", header=None)
    df = df.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    return df

# def sample(df,df_altr):
#     n=5
#     df = df.sample(n)
#     if df_altr.shape[0] > 100:
#         df_altr = df_altr.sample(n)
#     return df, df_altr
    

out = widgets.Output()
display(out)
def focused_eventhandler(change):
    os.chdir(r'C:\corpus\for display')
    global df,main,df_altr,flag
    if (change == 'ben yehuda'):
        main = pd.read_csv(r'over_view/ben yehuda.csv')
            
        fig = make_subplots(
            rows=2, cols=2,
            specs=[[{"type": "bar"}, {"type": "bar"}],
                    [{"type": "pie"}, {"type": "pie"}]],
            subplot_titles = ('authors','translators','Distribution of genres','Distribution of translated languages')
            )

        temp = over_view(main,'authors',10,' ','bar')
        fig.add_trace(go.Bar(y=temp['index'],
                                 x=temp['authors'],
                                orientation='h'),
                          row=1, col=1)

        temp = over_view(main[main['translators'].notnull()],'translators',10,'Translators','bar')
        fig.add_trace(go.Bar(y=temp['index'],
                                 x=temp['translators'],
                                orientation='h'),
                          row=1, col=2)

        temp = over_view(main,'genre',10,'Distribution of genres','pie')
        fig.add_trace(go.Pie(values=temp['genre'],
                            labels=temp['index']),
                        row=2, col=1)

        temp = over_view(main,'original_language',10,'Distribution of translated languages','pie')
        fig.add_trace(go.Pie(values=temp['original_language'],
                            labels=temp['index']),
                        row=2, col=2)

        fig.update_layout(height=700, showlegend=False)

        fig.show()
    elif (change == 'think il'):
        main = pd.read_csv(r'over_view/think il.csv')
        temp = over_view(main,'author',10,'Distribution of authors','bar')
        fig = go.Figure(
            data=[go.Bar(x=temp['author'], y=temp['index'], orientation='h')],
            layout=go.Layout(
                title=go.layout.Title(text="Distribution of authors")
            )
        )
        fig.show()
        
    elif (change == 'ParaShoot'):
        df = pd.read_csv('C:\corpus\ParaShoot\df.csv',sep = '\t')
        temp = df[['title']].value_counts().to_frame().reset_index()
        temp.columns = ['title','freq']
        fig = go.Figure(
            data=[go.Bar(y=temp['title'], x=temp['freq'], orientation='h')],
            layout=go.Layout(
                title=go.layout.Title(text="Distribution of article")
            )
        )
        fig.show()
        
    elif (change == 'Hebrew-Sentiment-Data'):
        df = pd.read_csv('C:\corpus\Hebrew-Sentiment-Data\dev.tsv',sep = '\t')
        df = df.rename(columns = {'comment':'line'})
        df = df['label'].value_counts().to_frame().reset_index()
        df.columns = ['lable','count']
        fig = go.Figure(go.Pie(values=df['count'],
                                    labels=['Positive','negative','neutral']))
        fig.show()
        
    
    elif (change == 'HeBERT Emotion Recognition'):
        df = pd.read_csv(r'C:\corpus\HeBERT Emotion Recognition\test_insample_tagged_raw_to_publish.csv')
        df = df.rename(columns = {'talkbacks':'line'})

        fig = make_subplots(
                            rows=1, cols=3,
                            specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]],
                    subplot_titles = ('Emotion Distribution','Sentiment Distribution','Source of response')
                    )
        temp = df['emotion_en'].value_counts().to_frame().reset_index()
        fig.add_trace(go.Pie(values=temp['emotion_en'],
                                    labels=temp['index']),
                                row=1, col=1)

        temp = df['value'].value_counts().to_frame().reset_index()
        fig.add_trace(go.Pie(values=temp['value'],
                                    labels=temp['index']),
                                row=1, col=2)

        temp = df['source'].value_counts().to_frame().reset_index()
        fig.add_trace(go.Pie(values=temp['source'],
                                    labels=temp['index']),
                                row=1, col=3)

        fig.update_layout(height=700, showlegend=False)
        fig.show()


def main_load_dataset(dirc: list):
    create_folder('over_view')
    #ben yehuda
    main = pd.read_csv(r'C:\corpus\ben yehuda\pseudocatalogue.csv')
    main[['authors','genre','original_language','translators']].to_csv(r'over_view/ben yehuda.csv')
    
    #think il
    main = pd.read_csv(r'C:\corpus\think il\df think IL token.tsv',sep = '\t')
    main = main.rename(columns={'text':'line','auther':'author'})
    main['author'].to_csv(r'over_view/think il.csv')

from plotly.subplots import make_subplots
focused_widget = widgets.Dropdown(options = ['ben yehuda','think il','ParaShoot','Hebrew-Sentiment-Data','HeBERT Emotion Recognition'],
                                  value = None,
                                  description='Chose data')
focused_view = widgets.interactive(focused_eventhandler, change=focused_widget)
focused_widget.observe(lambda x: focused_view.update(), 'value')
display(focused_view)